In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend as K
%matplotlib inline
import matplotlib.pyplot as plt

import glob
import os
import cv2
import sys
!pip install py7zr
import py7zr

In [ ]:
y_train = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')
y_test = pd.read_csv('/kaggle/input/cifar-10/sampleSubmission.csv')

y_train = np.array(y_train['label'])
y_test = np.array(y_test['label'])

y_test.shape

In [ ]:
!python -m py7zr x ../input/cifar-10/train.7z
!python -m py7zr x ../input/cifar-10/test.7z

In [ ]:
from PIL import Image

filelist = glob.glob("../working/train/*")
filelist = sorted(filelist,key= lambda x:int(x.split("/")[-1].split(".")[0]))
x_train = np.array([np.array(Image.open(fname)) for fname in filelist])
print(f"X.shape : {x_train.shape} y.shape : {y_train.shape}")

filelist = glob.glob("../working/test/*")
filelist = sorted(filelist,key= lambda x:int(x.split("/")[-1].split(".")[0]))
x_test = np.array([np.array(Image.open(fname)) for fname in filelist])
print(f"X.shape : {x_test.shape} y.shape : {y_test.shape}")

In [ ]:
#have a look on some images:

cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(x_train[random_index, :])
        ax.set_title([y_train[random_index]])
plt.show()

In [ ]:
#normalize inputs:
x_train2 = x_train/255 - 0.5
x_test2 = x_test/255 - 0.5

cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

y_train_indexes = []
for classs in y_train:
    y_train_indexes.append(cifar10_classes.index(classs))
    
# convert class labels to one-hot encoded:
y_train2 = keras.utils.to_categorical(y_train_indexes)

In [ ]:
# import necessary building blocks:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
#define CNN architecture:

def make_model():

    model = Sequential()

    model.add(Conv2D(filters = 16, kernel_size = (3, 3), padding='same', input_shape=(32, 32, 3)))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 32, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))
    
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), padding='same'))
    model.add(LeakyReLU(0.1))

    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))

    model.add(Dropout(0.5))
    
    model.add(Dense(10))
    model.add(Activation("softmax"))
    
    return model

In [ ]:
model = make_model()
model.summary()

In [ ]:
INIT_LR = 5e-3 
BATCH_SIZE = 32
EPOCHS = 10

model = make_model()

model.compile(
    loss='categorical_crossentropy',  
    optimizer=keras.optimizers.Adamax(lr=INIT_LR), 
    metrics=['accuracy']  # report accuracy during training
)

# scheduler of learning rate (decay with epochs)
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# callback for printing of actual learning rate used by optimizer
class LrHistory(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs={}):
        print("Learning rate:", K.get_value(model.optimizer.lr))

In [ ]:
# fit model:
import tensorflow_addons as tfa
tqdm_callback = tfa.callbacks.TQDMProgressBar()

model.fit(
    x_train2, y_train2,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(), 
               tqdm_callback],
    shuffle=True,
    verbose=0,
    initial_epoch=0
)

In [ ]:
# making test predictions:

x1=x_test2[0:75000]
x2=x_test2[75000:150000]
x3=x_test2[150000:225000]
x4=x_test2[225000:300000]

In [ ]:
y_pred_test_1 = model.predict(x1)
y_pred_test_classes_1 = np.argmax(y_pred_test_1, axis=1)
y_pred_test_classes_1

In [ ]:
y_pred_test_2 = model.predict(x2)
y_pred_test_classes_2 = np.argmax(y_pred_test_2, axis=1)
y_pred_test_classes_2

In [ ]:
y_pred_test_3 = model.predict(x3)
y_pred_test_classes_3 = np.argmax(y_pred_test_3, axis=1)
y_pred_test_classes_3

In [ ]:
y_pred_test_4 = model.predict(x4)
y_pred_test_classes_4 = np.argmax(y_pred_test_4, axis=1)
y_pred_test_classes_4

In [ ]:
y_pred = list(y_pred_test_classes_1) + list(y_pred_test_classes_2)+ list(y_pred_test_classes_3) + list(y_pred_test_classes_4)
len(y_pred)

In [ ]:
sample = pd.read_csv('/kaggle/input/cifar-10/sampleSubmission.csv')
sample

In [ ]:
y_pred_names = []

for ind in y_pred:
    y_pred_names.append(cifar10_classes[ind])
    
len(y_pred_names)

In [ ]:
#Saving results:

sample['label'] = y_pred_names
sample.to_csv("submission.csv", index=False)

sample.head()